<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from utils.attention_block import SelfAttention
from utils.prepare_data import read_file
from utils.prepare_data import get_vocab
from utils.prepare_data import train_validation_split
from utils.prepare_data import get_batch

# Set a seed for reproducibility
torch.manual_seed(0)

In [8]:
# Read text file, convert each token (char) into integers, and split data into train and validation data.
text = read_file('shakespeare.txt')

# Here are all the unique characters that occur in this text
vocab, vocab_size = get_vocab(text)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
train_data, val_data = train_validation_split(data, 0.9)

In [15]:
print(data.shape)

torch.Size([1115394])


In [107]:
# ------ Hyperparameters ------

# -- Data Breakdown
batch_size = 16
block_size = 8

# -- Network Parameters
# Embedding Block
num_embeddings = vocab_size
embedding_dim = 8

# Attention Block
kq_dim = 7
v_dim = 6

# -- Training Parameters
learning_rate = 3e-4
max_iters = 50000
eval_iters = 1000

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [51]:
# Both x and y are (batch_size, block_size)
x, y = get_batch(train_data, batch_size, block_size)

In [108]:
# Define Network

# Input Shape (batch_size, block_size). 
# Note block_size is not a parameters to the model, so it can accept arbitary
# length contexts.
model = nn.Sequential(
    # Transforms shape to (batch_size, block_size, embedding_dim)
    nn.Embedding(num_embeddings, embedding_dim),
    # Transforms shape to (batch_size, block_size, v_dim)
    SelfAttention(embedding_dim, kq_dim, v_dim),
    # Transforms shape to (batch_size, block_size, vocab_size)
    nn.Linear(v_dim, vocab_size)
    
)

print(model(x).shape)

torch.Size([4, 2, 65])


In [109]:
# -- Training

# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

def loss_fn(logits, targets):
    """
    logits: Shape (B, T, C)
    targets: Shape (B, T)
    """
    (B, T, C) = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)    
    # This wants logits to have have (N, C) and targets to have
    # shape (N,), where N is the batch size and C is the number of
    # classes.
    return F.cross_entropy(logits, targets)
    
@torch.no_grad()
def estimate_loss(train_data, val_data, eval_iters, batch_size, block_size):
    out = {}
    model.eval()
    for dataset_type, dataset in {'train': train_data, 'val': val_data}.items():
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(dataset, batch_size, block_size)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            losses[k] = loss.item()
        out[dataset_type] = losses.mean()
    model.train()
    return out

# Training Loop
for i in range(max_iters):
    # Shape = (batch_size, block_size)
    xb, yb = get_batch(train_data, batch_size, block_size)
    
    # Zero out the gradients
    optimizer.zero_grad()
    
    # Shape = (batch_size, block_size, vocab_size)
    logits = model(xb)
    
    # Compute loss
    loss = loss_fn(logits, yb)
    loss.backward()

    # Adjust weights
    optimizer.step()
    
    # Gather data report
    if i % eval_iters == 0 or i == max_iters - 1:
        eval_interval
        losses = estimate_loss(train_data, val_data, eval_iters, batch_size, block_size)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        

step <built-in function iter>: train loss 4.2920, val loss 4.2903
step <built-in function iter>: train loss 3.2538, val loss 3.2823
step <built-in function iter>: train loss 3.1020, val loss 3.1020
step <built-in function iter>: train loss 3.0278, val loss 3.0290
step <built-in function iter>: train loss 2.9686, val loss 2.9704
step <built-in function iter>: train loss 2.9241, val loss 2.9332
step <built-in function iter>: train loss 2.8800, val loss 2.8890
step <built-in function iter>: train loss 2.8357, val loss 2.8539
step <built-in function iter>: train loss 2.8181, val loss 2.8302
step <built-in function iter>: train loss 2.7954, val loss 2.8130
step <built-in function iter>: train loss 2.7824, val loss 2.8018
step <built-in function iter>: train loss 2.7631, val loss 2.7794
step <built-in function iter>: train loss 2.7501, val loss 2.7671
step <built-in function iter>: train loss 2.7382, val loss 2.7542
step <built-in function iter>: train loss 2.7269, val loss 2.7398
step <buil

In [110]:
def generate(model, context, max_context_size, max_new_tokens):
    """
    context: array tokens with shape (B, t), where t can be any length.
    """
    for _ in range(max_new_tokens):
        sub_context = context if context.shape[1] < max_context_size else context[:, -max_context_size:]
        # Shape = (batch_size, block_size, vocab_size)
        logits = model(sub_context)
        # Get the last character's predictions. Shape = (B, vocab_size)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1) 
        
        # Sample from the distribution. Shape = (B, 1)
        preds = torch.multinomial(probs, num_samples=1)   
        
        # Add to context
        context = torch.cat((context, preds), dim=1)
    return context
        
        

context = torch.zeros((1, 1), dtype=torch.long, device=device)
new_text = generate(model, context, block_size, max_new_tokens=500)
print(decode(new_text[0].tolist()))


CCRYDLBMEEOLwOOOR.GORxRRARMLPLCRSCEUONEZNULREENLRTULIAONEE ISICINNENE AR-IUL Ew:
ENLMSYsMELTnORy g


DAMORAUNAHRARBgNCRARCDYCARVDFEUOENUUSLE ETYLN:
FEwDBLINEETI:

TNRRJILI:
AIOR


HIIOR:
WTAShn
hwhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuurur.urursurrrrr.I.orrrfurrprnrrrl, lrr br Ioc t lfoab,,o tat, y feeotaaaaaoaaea aaat; t  it grr.o.orcsrprrrrxrrr!!!. rve!
Buvrm
Cenmtly s c f: f tsizt binhset I oct,oooo looo noo  toeofoo s totooo Voeooooooooooo eo y
